# Imports
Import required libraries: `requests`, `BeautifulSoup` (from `bs4`), `pandas`, and `time`.

In [10]:
# Import required libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# Base URL
Set `BASE_URL` to the Amazon search results page to scrape (no `&page=` parameter).

In [11]:
# Set the base Amazon search URL
BASE_URL = "https://www.amazon.in/s?k=playstation+5&crid=302ZJMTZG1JP1&sprefix=playstation+%2Caps%2C369&ref=nb_sb_noss_2"

# Request Headers
HTTP headers to mimic a browser; passed to `requests.get()` to reduce blocking.

In [12]:
# Set request headers to mimic a browser
HEADERS = {
    'user-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/143.0.0.0 Safari/537.36 Edg/143.0.0.0',
    'Accept-Language': 'en-US, en;q=0.5'
}

# Fetch Page Function
Defines `fetch_amazon_page(url, headers)` which returns a `BeautifulSoup` parsed page or `None` on failure.

In [13]:
# Function to fetch and parse a single Amazon search result page
def fetch_amazon_page(url, headers):
    try:
        response = requests.get(url, headers=headers, timeout=10)
        if response.status_code == 200:
            return BeautifulSoup(response.content, 'html.parser')
        else:
            print(f"Failed to fetch {url} (status code: {response.status_code})")
            return None
    except Exception as e:
        print(f"Error fetching {url}: {e}")
        return None

# Pagination Settings
Set `NUM_PAGES` for how many result pages to scrape and initialize `all_data` to collect results.

In [14]:
# Set how many pages to scrape and initialize data list
NUM_PAGES = 5  # Change this to scrape more/less pages
all_data = []

# Extract Product Data
Defines `extract_product_data(product)` to extract `Title`, `Price`, `Rating`, and `Link` from a product element and append to `all_data`.

In [16]:
# Extract product details from a single product element and append to all_data
def extract_product_data(product):
    title_elem = product.find('span', {'class': 'a-size-medium a-color-base a-text-normal'})
    if not title_elem:
        h2_elem = product.find('h2')
        if h2_elem:
            title_elem = h2_elem.find('span')
    title = title_elem.text.strip() if title_elem else None
    price_whole = product.find('span', {'class': 'a-price-whole'})
    price_fraction = product.find('span', {'class': 'a-price-fraction'})
    rating_elem = product.find('span', {'class': 'a-icon-alt'})
    link_elem = product.find('a', {'class': 'a-link-normal s-no-outline'})
    price = None
    if price_whole and price_fraction:
        price = price_whole.text.strip() + price_fraction.text.strip()
    elif price_whole:
        price = price_whole.text.strip()
    rating = rating_elem.text.strip() if rating_elem else None
    link = 'https://www.amazon.in' + link_elem['href'] if link_elem else None
    all_data.append({'Title': title, 'Price': price, 'Rating': rating, 'Link': link})

# Pagination Loop
Loop `page` from 1..`NUM_PAGES`, fetch each page, parse products, and collect data.

In [17]:
# Loop through multiple pages and collect product data
for page in range(1, NUM_PAGES + 1):
    paged_url = BASE_URL + f'&page={page}'
    print(f"Fetching page {page}...")
    soup = fetch_amazon_page(paged_url, HEADERS)
    if soup is None:
        continue
    products = soup.find_all('div', {'data-component-type': 's-search-result'})
    for product in products:
        extract_product_data(product)
    time.sleep(1)  # Be polite to Amazon's servers

Fetching page 1...
Fetching page 2...
Fetching page 3...
Fetching page 4...
Fetching page 5...


# Save Extracted Data
Convert `all_data` to a `pandas.DataFrame`, save to `amazon_products.csv`, and display the DataFrame.

In [18]:
# Save all multipage data to CSV and display
print(f"Total products scraped: {len(all_data)}")
df = pd.DataFrame(all_data)
df.to_csv('amazon_products.csv', index=False)
print('Saved all multipage results to amazon_products.csv')
df

Total products scraped: 110
Saved all multipage results to amazon_products.csv


,Title,Price,Rating,Link
0,Sage Controllers PRO+ Controller compatible wi...,"11,700",5.0 out of 5 stars,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
1,Sage Controllers PRO+ Controller compatible wi...,"14,000",5.0 out of 5 stars,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
2,Sony PlayStation5 Gaming Console (Slim),"54,990",4.5 out of 5 stars,https://www.amazon.in/Sony-CFI-2008A01X-PlaySt...
3,Sony PlayStation®5 Digital Edition (slim) Cons...,"49,990",4.6 out of 5 stars,https://www.amazon.in/Sony-PlayStation%C2%AE5-...
4,Sony DualSense Wireless Controller White (Play...,"4,890",4.2 out of 5 stars,https://www.amazon.in/DualSense-Wireless-Contr...
...,...,...,...,...
105,"ZORBES Dust Cover For Sony PS5 Console,Oxford ...",860,4.0 out of 5 stars,https://www.amazon.in/AntiScratch-Waterproof-P...
106,yantralay 6 in 1 Hard Carrying Case with Prote...,"2,070",4.6 out of 5 stars,https://www.amazon.in/yantralay-Carrying-Prote...
107,PowerA Ultra High Speed HDMI Cable 2.1 For Pla...,"1,999",4.6 out of 5 stars,https://www.amazon.in/PowerA-Ultra-HDMI-PlaySt...
108,GeekShare G.S.TAC Tactical PS5 Controller Skin...,"1,520",4.5 out of 5 stars,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
